In [1]:
from preprocessors.bilateral_filter import bilateral_filter
from preprocessors.gaussian_blur import gaussian_blur
from preprocessors.noise_reduction import denoise_image
from preprocessors.average_blur import layered_average_blur

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import json

import re
import ast

import cv2
import numpy as np
from skimage.restoration import denoise_nl_means, estimate_sigma
import matplotlib.pyplot as plt
import math
from preprocessors.bilateral_filter import bilateral_filter
from preprocessors.gaussian_blur import gaussian_blur
from preprocessors.noise_reduction import denoise_image
from preprocessors.average_blur import layered_average_blur
from preprocessors.isolate_intensity import isolate_intensity
from preprocessors.edge_detection import edge_detection
from preprocessors.hough_transform import hough_transform
import os
import pandas as pd

### Extract Coordinates

In [2]:
# Load png at file_path as a numpy array
def load_image(file_path):
    return np.asarray(cv2.imread(file_path, cv2.IMREAD_GRAYSCALE))

In [3]:
# Get all folders in data_path
def get_folders(data_path):
    return [os.path.join(data_path,f) for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

# Get all subfolders in data_path
def get_subfolders(data_path):
    return [os.path.join(data_path,f) for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

# Get all files in data_path
def get_files(data_path):
    # [os.path.join(data_path,f) for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.png')]
    all_files = []
    for f in os.listdir(data_path):
        if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.png'):
            all_files.append(os.path.join(data_path, f))

    # Get 10 equispaced files
    file_indexes = np.linspace(0, len(all_files)-1, 10, dtype=int)
    selected_files = [all_files[i] for i in file_indexes]
    return selected_files

In [4]:
# Get the 4 coordinates of the quadrilateral
def get_quadrilateral_coordinates(line_d, line_s, width):
    # Line is represented as (slope, intercept)
    
    slope_d, intercept_d = line_d
    slope_s, intercept_s = line_s

    # if nonetype return []
    if slope_d is None or intercept_d is None or slope_s is None or intercept_s is None:
        return []

    # Get the 4 points of the quadrilateral
    x1 = 0
    y1 = int(intercept_d)
    x2 = width
    y2 = int(slope_d * x2 + intercept_d)
    x3 = 0
    y3 = int(intercept_s)
    x4 = width
    y4 = int(slope_s * x4 + intercept_s)
    return [[x3, y3],[x4, y4], [x2, y2], [x1, y1]]

In [5]:
def extract_coordinates(frame_path, technique, method, threshold, flip=False):
    original_frame = load_image(frame_path)
    frame = original_frame.copy()
    if flip:
        frame = cv2.flip(frame, 1)
    if technique == 'gaussian':
        frame = gaussian_blur(frame)
    elif technique == 'bilateral':
        frame = bilateral_filter(frame)
    elif technique == 'average':
        frame = layered_average_blur(frame)
    elif technique == 'denoised':
        frame = denoise_image(frame)

    frame = isolate_intensity(frame, method)
    edges = edge_detection(frame)
    _, line_d, line_s = hough_transform(edges, threshold=threshold)
    if line_d is None or line_s is None:
        return []
    
    # if the technique is average, we need to scale the lines back from the down sampled image to the original image
    if technique == 'average':
        # Down sampling -> 1/5 -> 1/3
        # Get the shape of the original image
        original_width = load_image(frame_path).shape[1]
        # Down sampled width
        down_sampled_width = frame.shape[1]
        # Scale the lines back to the original image
        line_d = (line_d[0], line_d[1] * (original_width / down_sampled_width))
        line_s = (line_s[0], line_s[1] * (original_width / down_sampled_width))
        
    
    coordinates = get_quadrilateral_coordinates(line_d, line_s, original_frame.shape[1])
    #  set any negative coordinates to 0
    coordinates = [[max(0, x), max(0, y)] for x, y in coordinates]
    return coordinates


In [35]:
def get_coordinates_csv(data_path, csv_file_name='thresh_compare_avg.csv'):
    columns = ['Filename', 'Technique', 'Method', 'Threshold', 'Frame1', 'Frame2', 'Frame3', 'Frame4', 'Frame5', 'Frame6', 'Frame7', 'Frame8', 'Frame9', 'Frame10']
    if not os.path.exists(csv_file_name):
        pd.DataFrame(columns=columns).to_csv(csv_file_name, index=False)
    folders = get_folders(data_path)
    subfolders = []
    for folder in folders:
        subfolders += get_subfolders(folder)
    for subfolder in subfolders:
        files = get_files(subfolder)
        # Go through all 10 files
        # for technique in ['gaussian', 'bilateral', 'denoised']:
        for technique in ['average']:
            for method in ['q1', 'avg_min_intensity']:
                for threshold in range(1, 11):
                    print(f'Processing {subfolder} with technique {technique} and method {method} and threshold {threshold}')
                    frame_data = {'filename': subfolder, 'technique': technique, 'method': method, 'threshold': threshold}
                    for i, file in enumerate(files):
                        print(f'Processing frame {i+1}')
                        coordinates = extract_coordinates(file, technique, method, threshold, flip=True)
                        frame_data[f'Frame{i+1}'] = coordinates
                    # Append the data for this technique and method to the CSV
                    pd.DataFrame([frame_data]).to_csv(csv_file_name, mode='a', header=False, index=False)

In [36]:
data_path = 'PATH TO DATA'
get_coordinates_csv(data_path)

Processing /Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0 with technique average and method q1 and threshold 1
Processing frame 1
No lines found, reducing the angle range
Processing frame 2
No lines found, reducing the angle range
Processing frame 3
No lines found, reducing the angle range
Processing frame 4
No lines found, reducing the angle range
Processing frame 5
No lines found after y filtering in superficial transform.
Processing frame 6
No lines found after angle filtering in superficial transform.
Processing frame 7
No lines found, reducing the angle range
Processing frame 8
Processing frame 9
No lines found after angle filtering in superficial transform.
Processing frame 10
No lines found, reducing the angle range
Processing /Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0 with technique average and method q1 and threshold 2
Processing frame 1
Processing frame 2
No lines found, reducing the angle range
Processing frame 3
No lines found aft

### Extract IoU

In [9]:
def sort_points_clockwise(points):
    # Calculate the centroid of the points
    centroid = np.mean(points, axis=0)
    
    # Calculate the angles of each point with respect to the centroid
    def angle_with_centroid(point):
        delta = point - centroid
        return np.arctan2(delta[1], delta[0])
    
    # Sort the points by the angles
    sorted_points = sorted(points, key=angle_with_centroid)
    
    return np.array(sorted_points)

In [10]:
def read_roi(file_name, filepath, frame_num):
    df = pd.read_csv(filepath)
    file_name = file_name.split('.')[0]
    row = df.loc[df['Filename'] == file_name.split('.')[0]]
    pts = np.asarray((json.loads(row[f'Frame{frame_num}'].values[0])))

    pts = sort_points_clockwise(pts)

    return pts

def read_extracted_png(file_name, file_path, frame_num, technique, method, threshold):
    extracted_df = pd.read_csv(file_path)
    extracted_df['Filename'] = extracted_df['Filename'].apply(lambda x: x.split('.')[0])
    extracted_df = extracted_df[extracted_df['Filename'] == file_name.split('.')[0]]
    extracted_df = extracted_df[(extracted_df['Technique'] == technique) & (extracted_df['Method'] == method) & (extracted_df['Threshold'] == threshold)]
    
    array_str = extracted_df[f'Frame{frame_num}'].values[0]
    
    try:
        list_from_str = ast.literal_eval(array_str)
    except ValueError as e:
        print(array_str)
        print(f"Error converting string to list: {e}")
    
    extracted_roi = np.array(list_from_str)
    
    return extracted_roi

In [11]:
from shapely.geometry import Polygon
from shapely.validation import make_valid   

def calculate_IoU(roi, extracted):
    poly1 = Polygon(roi)
    poly2 = Polygon(extracted)

    poly1 = make_valid(poly1)
    poly2 = make_valid(poly2)

    # Calculate intersection and union areas
    try:
        intersection_area = poly1.intersection(poly2).area
        union_area = poly1.union(poly2).area
    except Exception as e:
        print(f"Error processing intersection or union: {e}")
        print(f"poly1: {poly1}")
        print(f"poly2: {poly2}")


    # Calculate IoU
    iou = intersection_area / union_area

    return iou, intersection_area, union_area

In [104]:
# Calculate IoU for all frames (1 to 10) and techniques and methods for a given file and save the results in a dataframe

def calculate_IoU_all_frames(file_name, roi_file_path, extracted_file_path):
    iou_df = pd.DataFrame(columns=['Filename','Frame', 'Technique', 'Method', 'Threshold', 'IoU', 'Intersection Area', 'Union Area'])
    rois = []
    for frame in range(1, 11):
        roi = read_roi(file_name, roi_file_path, frame)
        rois.append(roi)
    rois = sorted(rois, key=lambda x: Polygon(x).area)
    rois = np.array(rois)
    roi_median = rois[len(rois)//2]
    # for technique in ['denoised', 'bilateral', 'gaussian']:
    for technique in ['average']:
        for method in ['q1', 'avg_min_intensity']:
            for threshold in range(1, 11):
                extracts = []
                for frame in range(1, 11):
                    extracted = read_extracted_png(file_name, extracted_file_path, frame, technique, method, threshold)
                    if len(extracted) == 0:
                        continue
                    # if len(extracted) > 0:
                    extracts.append(extracted)

                if len(extracts) > 0:
                    extracts.sort(key=lambda x: Polygon(x).area)
                    extracts = np.array(extracts)
                    if len(extracts) == 0:
                        continue
                    extracted_q3 = np.percentile(extracts,50, axis=0)
                    roi_median = Polygon(roi_median).buffer(0)
                    extracted_q3 = Polygon(extracted_q3).buffer(0)
                    # print(extracted_q3)
                    iou, intersection_area, union_area = calculate_IoU(roi_median, extracted_q3)
                    iou_df.loc[len(iou_df)] = [file_name, frame, technique, method, threshold, iou, intersection_area, union_area]
    return iou_df

In [105]:
file_names = []
for folder in get_folders(data_path):
    file_names += get_subfolders(folder)

set(list(file_names))

{'/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_150_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_175_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_150_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_175_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_150_G0',


In [16]:
# Append dataframe to existing csv file
def append_to_csv(df, file_path):
    with open(file_path, 'a') as f:
        df.to_csv(f, header=f.tell()==0, index=False)

In [ ]:
for file_name in file_names:
    append_to_csv(calculate_IoU_all_frames(file_name, 'roi_data-png.csv', 'thresh_compare_avg.csv'), 'thresh_iou_avg.csv')

### Evaluate IoU

In [41]:
# Read iou_results.csv
iou_file_name = 'thresh_iou_avg.csv'
iou_results = pd.read_csv(iou_file_name)

In [42]:
# Number of duplicate rows in iou_results dataframe 
duplicate_rows = iou_results[iou_results.duplicated()]
duplicate_rows

,Filename,Frame,Technique,Method,Threshold,IoU,Intersection Area,Union Area


In [43]:
# # Remove duplicate rows from iou_results dataframe
iou_results = iou_results.drop_duplicates()
iou_results.to_csv(iou_file_name, index=False)

In [50]:
# For each technique, count how many times it appears in the iou_results dataframe
def count_technique_appearances(iou_results):
    technique_count_df = pd.DataFrame(columns=['Technique', 'Count'])
    for technique in ['average']:
        count = iou_results[iou_results['Technique'] == technique].shape[0]
        technique_count_df.loc[len(technique_count_df)] = [technique, count]
    return technique_count_df

In [24]:
count_technique_appearances(iou_results)

,Technique,Count
0,denoised,875
1,bilateral,869
2,gaussian,880


In [51]:
count_technique_appearances(iou_results)

,Technique,Count
0,average,874


In [48]:
# For each technique and method, calculate the average IoU across all frames and save the results in new dataframe
def calculate_average_iou(iou_results, technique):
    avg_iou_df = pd.DataFrame(columns=['Technique', 'Method', 'Threshold', 'Average IoU'])
    for method in ['q1', 'avg_min_intensity']:
        for threshold in range(1, 11):
            avg_iou = iou_results[(iou_results['Technique'] == technique) & (iou_results['Method'] == method) & (iou_results['Threshold'] == threshold)
                                ]['IoU'].mean()
            avg_iou_df.loc[len(avg_iou_df)] = [technique, method, threshold, avg_iou]
    return avg_iou_df

In [32]:
avg_iou = calculate_average_iou(iou_results, 'gaussian')
avg_iou.sort_values(by='Average IoU', ascending=False)

,Technique,Method,Threshold,Average IoU
10,gaussian,avg_min_intensity,10,0.661910
0,gaussian,q1,10,0.644866
1,gaussian,q1,20,0.634696
12,gaussian,avg_min_intensity,30,0.611648
2,gaussian,q1,30,0.606247
11,gaussian,avg_min_intensity,20,0.605361
3,gaussian,q1,40,0.588417
13,gaussian,avg_min_intensity,40,0.584961
4,gaussian,q1,50,0.571093
15,gaussian,avg_min_intensity,60,0.569138


In [33]:
avg_iou = calculate_average_iou(iou_results, 'bilateral')
avg_iou.sort_values(by='Average IoU', ascending=False)

,Technique,Method,Threshold,Average IoU
0,bilateral,q1,10,0.645204
1,bilateral,q1,20,0.644017
2,bilateral,q1,30,0.635777
3,bilateral,q1,40,0.631765
4,bilateral,q1,50,0.604981
11,bilateral,avg_min_intensity,20,0.581182
15,bilateral,avg_min_intensity,60,0.572923
10,bilateral,avg_min_intensity,10,0.568547
12,bilateral,avg_min_intensity,30,0.566375
5,bilateral,q1,60,0.565806


In [34]:
avg_iou = calculate_average_iou(iou_results, 'denoised')
avg_iou.sort_values(by='Average IoU', ascending=False)

,Technique,Method,Threshold,Average IoU
1,denoised,q1,20,0.645099
2,denoised,q1,30,0.632289
0,denoised,q1,10,0.629298
10,denoised,avg_min_intensity,10,0.610850
12,denoised,avg_min_intensity,30,0.608380
11,denoised,avg_min_intensity,20,0.605751
3,denoised,q1,40,0.603920
13,denoised,avg_min_intensity,40,0.594762
4,denoised,q1,50,0.579018
14,denoised,avg_min_intensity,50,0.572537


In [49]:
avg_iou = calculate_average_iou(iou_results, 'average')
avg_iou.sort_values(by='Average IoU', ascending=False)

,Technique,Method,Threshold,Average IoU
13,average,avg_min_intensity,4,0.700969
18,average,avg_min_intensity,9,0.690980
12,average,avg_min_intensity,3,0.687210
14,average,avg_min_intensity,5,0.680168
17,average,avg_min_intensity,8,0.674681
11,average,avg_min_intensity,2,0.672797
16,average,avg_min_intensity,7,0.668560
2,average,q1,3,0.662897
15,average,avg_min_intensity,6,0.661577
1,average,q1,2,0.660258


### Final Extraction

###### PNG

In [68]:
def get_coordinates_csv(data_path, csv_file_name='thresh_spec.csv'):
    columns = ['Filename', 'Technique', 'Method', 'Threshold', 'Frame1', 'Frame2', 'Frame3', 'Frame4', 'Frame5', 'Frame6', 'Frame7', 'Frame8', 'Frame9', 'Frame10']
    if not os.path.exists(csv_file_name):
        pd.DataFrame(columns=columns).to_csv(csv_file_name, index=False)
    folders = get_folders(data_path)
    subfolders = []
    for folder in folders:
        subfolders += get_subfolders(folder)
    for subfolder in subfolders:
        files = get_files(subfolder)
        # Go through all 10 files
        threshold_dict = {'gaussian': [10, 10], 'bilateral': [10, 20], 'denoised': [20, 10], 'average': [3, 4]}
        for technique in ['gaussian', 'bilateral', 'denoised', 'average']:
            for method in ['q1', 'avg_min_intensity']:
                threshold_index = 0 if method == 'q1' else 1
                threshold = threshold_dict[technique][threshold_index]
                print(f'Processing {subfolder} with technique {technique} and method {method} and threshold {threshold}')
                frame_data = {'filename': subfolder, 'technique': technique, 'method': method, 'threshold': threshold}
                for i, file in enumerate(files):
                    print(f'Processing frame {i+1}')
                    coordinates = extract_coordinates(file, technique, method, threshold, flip=True)
                    frame_data[f'Frame{i+1}'] = coordinates
                # Append the data for this technique and method to the CSV
                pd.DataFrame([frame_data]).to_csv(csv_file_name, mode='a', header=False, index=False)

In [69]:
data_path = 'YOUR DATA PATH'
get_coordinates_csv(data_path)

Processing /Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0 with technique gaussian and method q1 and threshold 10
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing /Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0 with technique gaussian and method avg_min_intensity and threshold 10
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing /Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0 with technique bilateral and method q1 and threshold 10
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing /

In [59]:
def read_extracted_png(file_name, file_path, frame_num, technique, method):
    extracted_df = pd.read_csv(file_path)
    extracted_df['Filename'] = extracted_df['Filename'].apply(lambda x: x.split('.')[0])
    extracted_df = extracted_df[extracted_df['Filename'] == file_name.split('.')[0]]
    extracted_df = extracted_df[(extracted_df['Technique'] == technique) & (extracted_df['Method'] == method)]
    
    array_str = extracted_df[f'Frame{frame_num}'].values[0]
    
    try:
        list_from_str = ast.literal_eval(array_str)
    except ValueError as e:
        print(array_str)
        print(f"Error converting string to list: {e}")
    
    extracted_roi = np.array(list_from_str)
    
    return extracted_roi

In [107]:
# Calculate IoU for all frames (1 to 10) and techniques and methods for a given file and save the results in a dataframe

def calculate_IoU_all_frames(file_name, roi_file_path, extracted_file_path):
    iou_df = pd.DataFrame(columns=['Filename','Frame', 'Technique', 'Method', 'IoU', 'Intersection Area', 'Union Area'])
    width = 512
    height = 512
    rois = []
    for frame in range(1, 11):
        roi = read_roi(file_name, roi_file_path, frame)
        rois.append(roi)
    rois = sorted(rois, key=lambda x: Polygon(x).area)
    rois = np.array(rois)
    roi_median = rois[len(rois)//2]
    for technique in ['gaussian', 'bilateral', 'denoised', 'average']:
        for method in ['q1', 'avg_min_intensity']:
            extracts = []
            for frame in range(1, 11):
                extracted = read_extracted_png(file_name, extracted_file_path, frame, technique, method)
                if len(extracted) == 0:
                    continue
                # if len(extracted) > 0:
                extracts.append(extracted)

            if len(extracts) > 0:
                extracts.sort(key=lambda x: Polygon(x).area)
                extracts = np.array(extracts)
                if len(extracts) == 0:
                    continue
                extracted_q3 = np.percentile(extracts, 75, axis=0)
                roi_median = Polygon(roi_median).buffer(0)
                extracted_q3 = Polygon(extracted_q3).buffer(0)
                # print(extracted_q3)
                iou, intersection_area, union_area = calculate_IoU(roi_median, extracted_q3)
                iou_df.loc[len(iou_df)] = [file_name, frame, technique, method, iou, intersection_area, union_area]
    return iou_df

In [108]:
file_names = []
for folder in get_folders(data_path):
    file_names += get_subfolders(folder)

set(list(file_names))

{'/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_150_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_175_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_150_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW128/WW128_175_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_100_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_G0',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_125_GP10',
 '/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW141/WW114_150_G0',


In [70]:
read_extracted_png('/Users/aarjavjain/Desktop/Education/Y4/PRJ/data/WW112/WW112_100_G0', 'thresh_spec.csv', 2, 'gaussian', 'q1')

array([[  0,  52],
       [512,  60],
       [512, 264],
       [  0, 371]])

In [109]:
for file_name in file_names:
    append_to_csv(calculate_IoU_all_frames(file_name, 'roi_data-png.csv', 'thresh_spec.csv'), 'thresh_iou_spec_75.csv')

In [130]:
# Read iou_results.csv
iou_file_name = 'thresh_iou_spec_75.csv'
iou_results = pd.read_csv(iou_file_name)

In [111]:
# Number of duplicate rows in iou_results dataframe 
duplicate_rows = iou_results[iou_results.duplicated()]
duplicate_rows

,Filename,Frame,Technique,Method,IoU,Intersection Area,Union Area


In [74]:
# # Remove duplicate rows from iou_results dataframe
iou_results = iou_results.drop_duplicates()
iou_results.to_csv(iou_file_name, index=False)


In [131]:
# For each technique, count how many times it appears in the iou_results dataframe
def count_technique_appearances(iou_results):
    technique_count_df = pd.DataFrame(columns=['Technique', 'Count'])
    for technique in ['gaussian', 'bilateral', 'denoised', 'average']:
        count = iou_results[iou_results['Technique'] == technique].shape[0]
        technique_count_df.loc[len(technique_count_df)] = [technique, count]
    return technique_count_df


In [132]:
count_technique_appearances(iou_results)

,Technique,Count
0,gaussian,88
1,bilateral,88
2,denoised,88
3,average,88


In [133]:
# For each technique and method, calculate the average IoU across all frames and save the results in new dataframe
def calculate_average_iou(iou_results):
    avg_iou_df = pd.DataFrame(columns=['Technique', 'Method', 'Average IoU'])
    for technique in ['gaussian', 'bilateral', 'denoised', 'average']:
        for method in ['q1', 'avg_min_intensity']:
                avg_iou = iou_results[(iou_results['Technique'] == technique) & (iou_results['Method'] == method)]['IoU'].mean()
                avg_iou_df.loc[len(avg_iou_df)] = [technique, method, avg_iou]
    return avg_iou_df


In [85]:
avg_iou = calculate_average_iou(iou_results)
avg_iou.sort_values(by='Average IoU', ascending=False)

,Technique,Method,Average IoU
7,average,avg_min_intensity,0.700969
6,average,q1,0.662897
1,gaussian,avg_min_intensity,0.661910
2,bilateral,q1,0.645204
4,denoised,q1,0.645099
0,gaussian,q1,0.644866
5,denoised,avg_min_intensity,0.610850
3,bilateral,avg_min_intensity,0.581182
